# Параллельные вычисления (2)

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Макрушин С.В. Лекция "Параллельные вычисления"
* https://nalepae.github.io/pandarallel/
    * https://github.com/nalepae/pandarallel/blob/master/docs/examples_windows.ipynb
    * https://github.com/nalepae/pandarallel/blob/master/docs/examples_mac_linux.ipynb
* https://requests.readthedocs.io/en/latest/
* https://docs.python.org/3/library/pathlib.html
* https://realpython.com/python-pathlib/
* https://realpython.com/python-gil/
* https://docs.python.org/3/library/multiprocessing.html#multiprocessing.pool.ThreadPool

## Задачи для совместного разбора

1. Выведите на экран слова из файла words_alpha, в которых есть две или более буквы "e" подряд.

In [1]:
import pandas as pd

words = (
    pd.read_csv("data/words_alpha.txt", header=None)[0]
    .dropna()
    .sample(frac=1, replace=True)
)

words

48656         cartonful
172440          limniad
226425      paretically
65554         condenses
61219            cobego
              ...      
87788          discuses
15625        antimonies
237530           piques
265244         reforger
190827    myzostomatous
Name: 0, Length: 370103, dtype: object

2. Загрузите данные о комментариях с сайта jsonplaceholder.typicode.com

![](https://i.imgur.com/AwiN8y6.png)

3. Получите множество уникальных почтовых доменов.

![](https://i.imgur.com/ceY6guU.png)

## Лабораторная работа 6

In [2]:
import pandas as pd
import re
import time
import numpy as np
from multiprocessing import Pool
import os
from functools import partial
import subprocess
import sys
from pandarallel import pandarallel
import requests
import time
from pathlib import Path
from tqdm.notebook import tqdm

__При решении данных задач не подразумевается использования циклов или генераторов Python в ходе работы с пакетами `numpy` и `pandas`, если в задании не сказано обратного. Решения задач, в которых для обработки массивов `numpy` или структур `pandas` используются явные циклы (без согласования с преподавателем), могут быть признаны некорректными и не засчитаны.__

<p class="task" id="1"></p>

1\. Напишите функцию `f`, которая принимает на вход тэг и проверяет, удовлетворяет ли тэг следующему шаблону: `[любое число]-[любое слово]-or-less`. Возьмите файл `tag_nsteps_10m.csv`, примените функцию `f` при помощи метода _серий_ `map` к столбцу `tags` и посчитайте количество тэгов, подходящих под этот шаблон. Выведите количество подходящих тегов на экран. Измерьте время выполнения кода.

In [3]:
%%time

def f(tag: str) -> bool:
    """
    Проверяет, удовлетворяет ли тэг шаблону: [любое число]-[любое слово]-or-less.
    """
    import re
    if not isinstance(tag, str):
        return False
    pattern = r'^\d+-[a-zA-Z]+-or-less$'
    return bool(re.fullmatch(pattern, tag))

df = pd.read_csv('data/tag_nsteps_10m.csv')
matching_count = df['tags'].map(f).sum()

print(f"Количество подходящих тегов: {matching_count}")

Количество подходящих тегов: 288503
CPU times: total: 6.7 s
Wall time: 7.1 s


<p class="task" id="2"></p>

2\. Напишите функцию `parallel_map`, которая принимает на вход серию `s` `pd.Series` и функцию одного аргумента `f` и поэлементно применяет эту функцию к серии, распараллелив вычисления при помощи пакета `multiprocessing`. Логика работы функции `parallel_map` должна включать следующие действия:
* разбиение исходной серии на $K$ частей, где $K$ - количество ядер вашего процессора;
* параллельное применение функции `f` к каждой части при помощи метода _серии_ `map` c использованием нескольких подпроцессов;
* объединение результатов работы подпроцессов в одну серию. 

Возьмите файл `tag_nsteps_10m.csv`, примените функцию `f` при помощи `parallel_map` к столбцу `tags` и посчитайте количество тэгов, подходящих под этот шаблон. Выведите количество подходящих тегов на экран. Измерьте время выполнения кода.

In [4]:
%%writefile 06.2.py

import pandas as pd
import re
import time
import numpy as np
from multiprocessing import Pool
import os
from functools import partial

def f(tag: str) -> bool:
    """
    Проверяет, удовлетворяет ли тэг шаблону: [любое число]-[любое слово]-or-less.
    """
    if not isinstance(tag, str):
        return False
    pattern = r'^\d+-[a-zA-Z]+-or-less$'
    return bool(re.fullmatch(pattern, tag))

def _map_on_chunk(chunk: pd.Series, func: callable) -> pd.Series:
    """Вспомогательная функция для применения .map() к части данных."""
    return chunk.map(func)

def parallel_map(s: pd.Series, f: callable) -> pd.Series:
    """
    Применяет функцию f к серии s, распараллеливая вычисления
    на K процессов, где K - количество ядер CPU.
    """
    cpu_cores = os.cpu_count()
    if cpu_cores is None:
        cpu_cores = 2
    print(f"Используется {cpu_cores} ядер процессора.")

    chunks = np.array_split(s, cpu_cores)
    
    worker_func = partial(_map_on_chunk, func=f)

    with Pool(cpu_cores) as pool:
        processed_chunks = pool.map(worker_func, chunks)

    result_series = pd.concat(processed_chunks)
    
    return result_series

if __name__ == '__main__':
    start_time = time.time()

    df = pd.read_csv('\\'.join(__file__.split('\\')[:-1])+'\\data\\tag_nsteps_10m.csv')

    matching_series = parallel_map(df['tags'], f)

    matching_count = matching_series.sum()

    print(f"Количество подходящих тегов: {matching_count}")

    end_time = time.time()
    
    print(f"Время выполнения: {end_time - start_time:.4f} секунд")

Overwriting 06.2.py


In [5]:
def run_file(app_file):
    """Запускает python-скрипт и возвращает его stdout и stderr."""
    
    command = [sys.executable, app_file]

    process = subprocess.Popen(
        command,
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True,
        encoding='utf-8'
    )
    
    stdout, stderr = process.communicate()
    
    # Печатаем результат
    print("--- STDOUT ---")
    print(stdout)
    
    if stderr:
        print("--- STDERR ---")
        print(stderr)

run_file('06.2.py')

--- STDOUT ---
Используется 16 ядер процессора.
Количество подходящих тегов: 288503
Время выполнения: 4.0192 секунд

--- STDERR ---
c:\Projects\FU\.venv\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)



<p class="task" id="3"></p>

3\. Используя пакет `pandarallel`, примените функцию `f` из задания 1 к столбцу `tags` таблицы, с которой вы работали в этом задании. Посчитайте количество тэгов, подходящих под описанный шаблон. Выведите на экран полученный результат. Измерьте время выполнения кода. 

In [6]:
%%time

pandarallel.initialize(progress_bar=True, verbose=2)


df = pd.read_csv('data/tag_nsteps_10m.csv')

matching_count = df['tags'].parallel_apply(f).sum()

print(f"Количество подходящих тегов: {matching_count}")

INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


Количество подходящих тегов: 288503
CPU times: total: 2.19 s
Wall time: 5.25 s


<p class="task" id="4"></p>

4\. Сайт [DummyJSON](https://dummyjson.com/) позволяет получить набор данных о товарах в виде JSON. Воспользовавшись пакетом `requests`, получите данные о __50 товарах__ и создайте словарь, где ключом является название товара (title), а значением - список ссылок на изображения этого товара. При создании словаря замените символ `/` в названии на пробел.

Выведите на экран количество элементов полученного словаря.

In [7]:
start_time = time.time()

url = "https://dummyjson.com/products?limit=50&skip=0"

try:
    response = requests.get(url)
    response.raise_for_status()
    data = response.json()

    products_dict = {}

    if 'products' in data:
        for product in data['products']:
            title = product.get('title')
            images = product.get('images', [])

            if title and images:
                formatted_title = title.replace('/', ' ')
                
                products_dict[formatted_title] = images
    else:
        print("Ошибка: Ответ API не содержит ключ 'products'.")

    print(f"Количество товаров в словаре: {len(products_dict)}")

except requests.exceptions.RequestException as e:
    print(f"Ошибка при запросе к API: {e}")
except KeyError as e:
    print(f"Ошибка при обработке JSON: отсутствует ожидаемый ключ - {e}")
except Exception as e:
    print(f"Произошла непредвиденная ошибка: {e}")

end_time = time.time()
print(f"Время выполнения: {end_time - start_time:.4f} секунд")

Количество товаров в словаре: 50
Время выполнения: 1.0226 секунд


<p class="task" id="5"></p>

5\. Напишите функцию `download_product_imgs`, которая создает папку c названием товара внутри каталога `imgs` (сам каталог `imgs` может быть создан любым удобным способом до начала работы) и сохраняет в нее изображения. Название для файла изображения извлеките из URL.

Воспользовавшись этой функцией, скачайте изображения всех продуктов. Выведите на экран общее количество загруженных файлов. Для отслеживания хода выполнения кода используйте пакет `tqdm`.

In [8]:
# пример кода для скачивания картинки
url = "https://png.pngtree.com/png-vector/20201229/ourmid/pngtree-a-british-short-blue-and-white-cat-png-image_2654518.jpg"
img = requests.get(url).content
with open("cat.jpg", "wb") as fp:
    fp.write(img)

In [9]:
def download_product_imgs(title, imgs):
    '''
    title - название товара
    imgs - список ссылок на изображения товара
    '''
    pass

In [10]:
root_imgs_dir = Path("imgs")
root_imgs_dir.mkdir(exist_ok=True)

In [11]:
def download_product_imgs(title: str, img_urls: list):
    """
    Создает папку для товара и скачивает в нее все изображения.
    
    Args:
        title (str): Название товара, используется для имени папки.
        img_urls (list): Список URL-адресов изображений для скачивания.
    """
    product_dir = root_imgs_dir / title
    product_dir.mkdir(exist_ok=True)
    
    for url in img_urls:
        try:
            img_content = requests.get(url).content
            
            file_name = Path(url).name
            
            save_path = product_dir / file_name
            
            with open(save_path, "wb") as f:
                f.write(img_content)
        except requests.exceptions.RequestException as e:
            print(f"Не удалось скачать {url}: {e}")

In [12]:
print("Начинаю скачивание изображений...")
start_time = time.time()

for title, imgs in tqdm(products_dict.items(), desc="Скачивание продуктов"):
    download_product_imgs(title, imgs)

end_time = time.time()
print("Скачивание завершено.")
print(f"Время выполнения: {end_time - start_time:.4f} секунд")
total_files = sum(1 for path in root_imgs_dir.rglob('*') if path.is_file())

print(f"Общее количество загруженных файлов: {total_files}")

Начинаю скачивание изображений...


Скачивание продуктов:   0%|          | 0/50 [00:00<?, ?it/s]

Скачивание завершено.
Время выполнения: 93.6264 секунд
Общее количество загруженных файлов: 83


<p class="task" id="6"></p>

6\. Создайте функцию `download_product_imgs_processes` на основе функции `download_product_imgs`, добавив в нее вывод сообщения следующего вида: `Process ID: <ID текущего процесса>`. Для определения ID процесса воспользуйтесь функцией `multiprocessing.current_process()`.

Решите задачу 5, распараллелив вычисления при помощи процессов. Вместо корневого каталога `imgs` используйте `imgs_processes`. Выведите на экран общее количество загруженных файлов. Измерьте время выполнения кода. 

In [13]:
%%writefile 06.6.py

import requests
from pathlib import Path
from tqdm import tqdm
import time
import multiprocessing
import os

url = "https://dummyjson.com/products?limit=50&skip=0"
products_dict = {}
try:
    response = requests.get(url)
    response.raise_for_status()
    data = response.json()
    if 'products' in data:
        for product in data['products']:
            title = product.get('title')
            images = product.get('images', [])
            if title and images:
                formatted_title = title.replace('/', ' ')
                products_dict[formatted_title] = images
except requests.exceptions.RequestException as e:
    print(f"Не удалось получить данные о продуктах: {e}")
    exit()


ROOT_IMGS_DIR_PROCESSES = Path("imgs_processes")

def download_product_imgs_processes(args):
    """
    Создает папку для товара, скачивает в нее все изображения
    и выводит ID текущего процесса.
    Принимает кортеж (title, img_urls) для совместимости с pool.map.
    """
    title, img_urls = args
    
    # Выводим ID процесса
    process_id = multiprocessing.current_process().pid
    print(f"Process ID: {process_id} | Скачиваю изображения для '{title}'")
    
    # Создаем папку для конкретного товара
    product_dir = ROOT_IMGS_DIR_PROCESSES / title
    product_dir.mkdir(exist_ok=True, parents=True)
    
    for url in img_urls:
        try:
            img_content = requests.get(url).content
            file_name = Path(url).name
            save_path = product_dir / file_name
            with open(save_path, "wb") as f:
                f.write(img_content)
        except requests.exceptions.RequestException as e:
            print(f"Process ID: {process_id} | Не удалось скачать {url}: {e}")


if __name__ == '__main__':
    # Создаем корневую директорию
    ROOT_IMGS_DIR_PROCESSES.mkdir(exist_ok=True)
    
    print("Начинаю параллельное скачивание изображений с помощью процессов...")
    start_time = time.time()

    tasks = list(products_dict.items())
    
    num_processes = os.cpu_count() or 2
    print(f"Используется {num_processes} процессов.")
    
    with multiprocessing.Pool(processes=num_processes) as pool:
        list(tqdm(pool.imap(download_product_imgs_processes, tasks), total=len(tasks), desc="Скачивание (процессы)"))

    end_time = time.time()
    print("\nСкачивание завершено.")
    print(f"Время выполнения: {end_time - start_time:.4f} секунд")

    total_files = sum(1 for path in ROOT_IMGS_DIR_PROCESSES.rglob('*') if path.is_file())
    print(f"\nОбщее количество загруженных файлов: {total_files}")

Overwriting 06.6.py


In [14]:
run_file('06.6.py')

--- STDOUT ---
Начинаю параллельное скачивание изображений с помощью процессов...
Используется 16 процессов.
Process ID: 23612 | Скачиваю изображения для 'Essence Mascara Lash Princess'
Process ID: 30416 | Скачиваю изображения для 'Eyeshadow Palette with Mirror'
Process ID: 21476 | Скачиваю изображения для 'Powder Canister'
Process ID: 2536 | Скачиваю изображения для 'Red Lipstick'
Process ID: 10248 | Скачиваю изображения для 'Red Nail Polish'
Process ID: 10876 | Скачиваю изображения для 'Calvin Klein CK One'
Process ID: 36532 | Скачиваю изображения для 'Chanel Coco Noir Eau De'
Process ID: 29308 | Скачиваю изображения для 'Dior J'adore'
Process ID: 23676 | Скачиваю изображения для 'Dolce Shine Eau de'
Process ID: 23668 | Скачиваю изображения для 'Gucci Bloom Eau de'
Process ID: 33112 | Скачиваю изображения для 'Annibale Colombo Bed'
Process ID: 40436 | Скачиваю изображения для 'Annibale Colombo Sofa'
Process ID: 8100 | Скачиваю изображения для 'Bedside Table African Cherry'
Process ID

<p class="task" id="7"></p>

7\. Создайте функцию `download_product_imgs_threads` на основе функции `download_product_imgs`, добавив в нее вывод сообщения следующего вида: `Process ID: <ID текущего процесса> Thread ID: <ID текущего потока>`. Для определения ID потока воспользуйтесь функцией `threading.get_ident`.

Решите задачу 5, распараллелив вычисления при помощи потоков. Вместо корневого каталога `imgs` используйте `imgs_threads`. Выведите на экран общее количество загруженных файлов. Измерьте время выполнения кода. 

In [15]:
%%writefile 06.7.py

import requests
from pathlib import Path
from tqdm import tqdm
import time
import os
import threading
from multiprocessing.pool import ThreadPool 

url = "https://dummyjson.com/products?limit=50&skip=0"
products_dict = {}
try:
    response = requests.get(url)
    response.raise_for_status()
    data = response.json()
    if 'products' in data:
        for product in data['products']:
            title = product.get('title')
            images = product.get('images', [])
            if title and images:
                formatted_title = title.replace('/', ' ')
                products_dict[formatted_title] = images
except requests.exceptions.RequestException as e:
    print(f"Не удалось получить данные о продуктах: {e}")
    exit()


ROOT_IMGS_DIR_THREADS = Path("imgs_threads")

def download_product_imgs_threads(args):
    """
    Создает папку для товара, скачивает в нее все изображения
    и выводит ID текущего процесса и потока.
    Принимает кортеж (title, img_urls) для совместимости с pool.map.
    """
    title, img_urls = args
    
    # Получаем ID процесса и потока
    process_id = os.getpid() # os.getpid() - простой способ получить ID процесса
    thread_id = threading.get_ident()
    
    # Выводим ID процесса и потока
    print(f"Process ID: {process_id} Thread ID: {thread_id} | Скачиваю изображения для '{title}'")
    
    # Создаем папку для конкретного товара
    product_dir = ROOT_IMGS_DIR_THREADS / title
    product_dir.mkdir(exist_ok=True, parents=True)
    
    for url in img_urls:
        try:
            img_content = requests.get(url).content
            file_name = Path(url).name
            save_path = product_dir / file_name
            with open(save_path, "wb") as f:
                f.write(img_content)
        except requests.exceptions.RequestException as e:
            print(f"Process ID: {process_id} Thread ID: {thread_id} | Не удалось скачать {url}: {e}")

# --- Шаг 3: Параллельное скачивание с использованием ThreadPool ---

# Создаем корневую директорию
ROOT_IMGS_DIR_THREADS.mkdir(exist_ok=True)

print("Начинаю параллельное скачивание изображений с помощью потоков...")
start_time = time.time()

# Подготавливаем список задач для пула потоков
tasks = list(products_dict.items())

# Определяем количество потоков (для I/O-задач можно использовать больше потоков, чем ядер)
num_threads = os.cpu_count() or 2
print(f"Используется {num_threads} потоков.")

# Создаем пул потоков и распределяем задачи
# Интерфейс ThreadPool идентичен Pool
with ThreadPool(processes=num_threads) as pool:
    # Используем pool.imap для совместимости с tqdm
    list(tqdm(pool.imap(download_product_imgs_threads, tasks), total=len(tasks), desc="Скачивание (потоки)"))

end_time = time.time()
print("\nСкачивание завершено.")
print(f"Время выполнения: {end_time - start_time:.4f} секунд")

# --- Шаг 4: Подсчет общего количества загруженных файлов ---
total_files = sum(1 for path in ROOT_IMGS_DIR_THREADS.rglob('*') if path.is_file())
print(f"\nОбщее количество загруженных файлов: {total_files}")

Overwriting 06.7.py


In [16]:
run_file('06.7.py')

--- STDOUT ---
Начинаю параллельное скачивание изображений с помощью потоков...
Используется 16 потоков.
Process ID: 34540 Thread ID: 16276 | Скачиваю изображения для 'Essence Mascara Lash Princess'Process ID: 34540 Thread ID: 42956 | Скачиваю изображения для 'Eyeshadow Palette with Mirror'Process ID: 34540 Thread ID: 37704 | Скачиваю изображения для 'Powder Canister'

Process ID: 34540 Thread ID: 8888 | Скачиваю изображения для 'Red Lipstick'

Process ID: 34540 Thread ID: 38036 | Скачиваю изображения для 'Red Nail Polish'
Process ID: 34540 Thread ID: 36936 | Скачиваю изображения для 'Calvin Klein CK One'
Process ID: 34540 Thread ID: 28800 | Скачиваю изображения для 'Chanel Coco Noir Eau De'Process ID: 34540 Thread ID: 21360 | Скачиваю изображения для 'Dior J'adore'Process ID: 34540 Thread ID: 34132 | Скачиваю изображения для 'Dolce Shine Eau de'
Process ID: 34540 Thread ID: 27164 | Скачиваю изображения для 'Gucci Bloom Eau de'Process ID: 34540 Thread ID: 12680 | Скачиваю изображения д

<p class="task" id="8"></p>

8\. Напишите функцию `create_2d_list`, которая создает матрицу размера `m` на `n` (__в виде списка списков__) вещественных чисел из стандартного нормального распределения. Напишите функцию `sum_by_chunk`, которая принимает на вход несколько строк этой матрицы (тоже в виде списка списков) и находит сумму элементов. 

Используя данную функцию, решите задачу поиска суммы по всей матрице тремя способами:
* передав в функцию `sum_by_chunk` всю матрицу целиком;
* распараллелив вычисления при помощи процессов по следующему принципу: матрица разбивается на части (например, по 1тыс. строк); процессы независимо друг от друга обрабатывают эти части; после завершения работы всех процессов результаты агрегируются для получения результата для всей матрицы;
* распараллелив вычисления при помощи потоков аналогичным способом.

Для демонстрации результата создайте матрицу достаточно большого размера (не менее 100 тыс. строк), выведите на экран результаты работы трех вариантов решения и измерьте время выполнения каждого из них.

В данном задании разрешается использовать пакет `numpy` только для создания матрицы. В этом случае необходимо преобразовать ее к списку списков до начала работы.

In [17]:
%%writefile 06.8.py

import numpy as np
import time
import os
import multiprocessing
from multiprocessing.pool import ThreadPool

# --- Шаг 1: Определение функций ---

def create_2d_list(m: int, n: int) -> list[list[float]]:
    """
    Создает матрицу m x n в виде списка списков,
    заполненную числами из стандартного нормального распределения.
    """
    # Используем numpy для быстрой генерации, затем конвертируем в list of lists
    return np.random.randn(m, n).tolist()

def sum_by_chunk(chunk: list[list[float]]) -> float:
    """
    Принимает часть матрицы (список списков) и возвращает сумму ее элементов.
    Использует стандартные средства Python, без numpy.
    """
    # Используем генераторное выражение для эффективности и краткости
    return sum(item for row in chunk for item in row)

# --- Шаг 2: Основной блок выполнения ---

# Обязательная конструкция для использования multiprocessing
if __name__ == '__main__':
    # Задаем параметры
    M = 200_000  # Количество строк
    N = 100      # Количество столбцов
    CHUNK_SIZE = 1000 # Размер одной части для параллельной обработки

    print(f"Создается матрица {M}x{N} в виде списка списков...")
    matrix = create_2d_list(M, N)
    print("Матрица создана.\n")

    # --- 1. Последовательное выполнение ---
    print("--- 1. Последовательное выполнение ---")
    start_time_seq = time.time()
    
    total_sum_seq = sum_by_chunk(matrix)
    
    end_time_seq = time.time()
    print(f"Сумма элементов: {total_sum_seq:.4f}")
    print(f"Время выполнения: {end_time_seq - start_time_seq:.4f} секунд\n")

    # --- 2. Параллельное выполнение с процессами ---
    print("--- 2. Параллельное выполнение с процессами ---")
    start_time_proc = time.time()
    
    # Разбиваем матрицу на части
    chunks = [matrix[i:i + CHUNK_SIZE] for i in range(0, M, CHUNK_SIZE)]
    
    num_processes = os.cpu_count() or 2
    print(f"Используется {num_processes} процессов...")
    
    with multiprocessing.Pool(processes=num_processes) as pool:
        # Распределяем chunks по процессам и применяем функцию sum_by_chunk
        results_proc = pool.map(sum_by_chunk, chunks)
    
    # Агрегируем результаты
    total_sum_proc = sum(results_proc)
    
    end_time_proc = time.time()
    print(f"Сумма элементов: {total_sum_proc:.4f}")
    print(f"Время выполнения: {end_time_proc - start_time_proc:.4f} секунд\n")

    # --- 3. Параллельное выполнение с потоками ---
    print("--- 3. Параллельное выполнение с потоками ---")
    start_time_thread = time.time()
    
    # chunks у нас уже есть из предыдущего шага
    num_threads = os.cpu_count() or 2
    print(f"Используется {num_threads} потоков...")
    
    with ThreadPool(processes=num_threads) as pool:
        # Распределяем chunks по потокам
        results_thread = pool.map(sum_by_chunk, chunks)
        
    # Агрегируем результаты
    total_sum_thread = sum(results_thread)
    
    end_time_thread = time.time()
    print(f"Сумма элементов: {total_sum_thread:.4f}")
    print(f"Время выполнения: {end_time_thread - start_time_thread:.4f} секунд\n")

Overwriting 06.8.py


In [18]:
run_file('06.8.py')

--- STDOUT ---
Создается матрица 200000x100 в виде списка списков...
Матрица создана.

--- 1. Последовательное выполнение ---
Сумма элементов: -2974.9576
Время выполнения: 0.6711 секунд

--- 2. Параллельное выполнение с процессами ---
Используется 16 процессов...
Сумма элементов: -2974.9576
Время выполнения: 1.1951 секунд

--- 3. Параллельное выполнение с потоками ---
Используется 16 потоков...
Сумма элементов: -2974.9576
Время выполнения: 0.6519 секунд




<p class="task" id="9"></p>

9\. Напишите функцию `create_2d_arr`, которая создает матрицу размера `m` на `n` (__в виде массива numpy__) вещественных чисел из стандартного нормального распределения. Напишите функцию `sum_by_chunk_np`, которая принимает на вход несколько строк этой матрицы (тоже в виде массива) и находит сумму элементов. 

Используя данную функцию, решите задачу поиска суммы по всей матрице тремя способами аналогично задаче 8.

Для демонстрации результата создайте матрицу достаточно большого размера (не менее 100 тыс. строк), выведите на экран результаты работы трех вариантов решения и измерьте время выполнения каждого из них.

В данном задании при поиска суммы не используйте встроенную функцию `sum`, вместо этого используйте возможности `numpy`.

In [19]:
%%writefile 06.9.py

import numpy as np
import time
import os
import multiprocessing
from multiprocessing.pool import ThreadPool

# --- Шаг 1: Определение функций для NumPy ---

def create_2d_arr(m: int, n: int) -> np.ndarray:
    """
    Создает матрицу m x n в виде массива numpy,
    заполненную числами из стандартного нормального распределения.
    """
    return np.random.randn(m, n)

def sum_by_chunk_np(chunk: np.ndarray) -> float:
    """
    Принимает часть матрицы (массив numpy) и возвращает сумму ее элементов,
    используя np.sum().
    """
    return np.sum(chunk)

# --- Шаг 2: Основной блок выполнения ---

# Обязательная конструкция для использования multiprocessing
if __name__ == '__main__':
    # Задаем параметры
    M = 200_000  # Количество строк
    N = 100      # Количество столбцов
    CHUNK_SIZE = 1000 # Размер одной части для параллельной обработки

    print(f"Создается матрица {M}x{N} в виде массива numpy...")
    matrix_np = create_2d_arr(M, N)
    print("Матрица создана.\n")

    # --- 1. Последовательное выполнение (Numpy) ---
    print("--- 1. Последовательное выполнение (Numpy) ---")
    start_time_seq = time.time()
    
    total_sum_seq = sum_by_chunk_np(matrix_np)
    
    end_time_seq = time.time()
    print(f"Сумма элементов: {total_sum_seq:.4f}")
    print(f"Время выполнения: {end_time_seq - start_time_seq:.4f} секунд\n")

    # --- 2. Параллельное выполнение с процессами (Numpy) ---
    print("--- 2. Параллельное выполнение с процессами (Numpy) ---")
    start_time_proc = time.time()
    
    # Разбиваем массив numpy на части
    chunks = [matrix_np[i:i + CHUNK_SIZE] for i in range(0, M, CHUNK_SIZE)]
    
    num_processes = os.cpu_count() or 2
    print(f"Используется {num_processes} процессов...")
    
    with multiprocessing.Pool(processes=num_processes) as pool:
        results_proc = pool.map(sum_by_chunk_np, chunks)
    
    total_sum_proc = sum(results_proc)
    
    end_time_proc = time.time()
    print(f"Сумма элементов: {total_sum_proc:.4f}")
    print(f"Время выполнения: {end_time_proc - start_time_proc:.4f} секунд\n")

    # --- 3. Параллельное выполнение с потоками (Numpy) ---
    print("--- 3. Параллельное выполнение с потоками (Numpy) ---")
    start_time_thread = time.time()
    
    # chunks у нас уже есть
    num_threads = os.cpu_count() or 2
    print(f"Используется {num_threads} потоков...")
    
    with ThreadPool(processes=num_threads) as pool:
        results_thread = pool.map(sum_by_chunk_np, chunks)
        
    total_sum_thread = sum(results_thread)
    
    end_time_thread = time.time()
    print(f"Сумма элементов: {total_sum_thread:.4f}")
    print(f"Время выполнения: {end_time_thread - start_time_thread:.4f} секунд\n")

Overwriting 06.9.py


In [20]:
run_file('06.9.py')

--- STDOUT ---
Создается матрица 200000x100 в виде массива numpy...
Матрица создана.

--- 1. Последовательное выполнение (Numpy) ---
Сумма элементов: -409.3581
Время выполнения: 0.0258 секунд

--- 2. Параллельное выполнение с процессами (Numpy) ---
Используется 16 процессов...
Сумма элементов: -409.3581
Время выполнения: 1.0240 секунд

--- 3. Параллельное выполнение с потоками (Numpy) ---
Используется 16 потоков...
Сумма элементов: -409.3581
Время выполнения: 0.0134 секунд


